In [ ]:
import sys
sys.path.append('~/PycharmProjects/code-trans-aug/')
storage_root = '~/PycharmProjects/code-trans-aug/storage/'

import os, ast
import pandas as pd
import numpy as np
#!pip install javalang
import javalang
import javalang.tree
import javalang.ast
import javalang.util
from javalang.ast import Node
import pandas as pd

def remove_unnecessary(dr):
    for (rt, dr, files) in os.walk(path, topdown=True):
        for file in files:
            ext = file.split('.')[1]
            if ext != 'java' and ext != 'py':
                print(rt+'/'+file)
                os.remove(rt+'/'+file)
                #continue
                
def read(f_path):
    text = -1
    try:
        text = open(f_path).read().strip()
    except:
        erroneous_files.append([f_path, ext])
    return text
def is_parsable(f_path, ext):
    text = read(f_path)
    s = f_path.split("/")
    fp = "/".join(s[2:])
    pc= "/".join(s[2:-1])
    #print(fp, pc)
    
    if text != -1:
        if ext=='py':
            try:
                _ = ast.parse(text)
                dataset.append([fp, pc, text, ext, 1])
            except:
                dataset.append([fp, pc, text, ext, 0])
        elif ext=='java':
            try:
                programtokens = javalang.tokenizer.tokenize(text)
                parser = javalang.parse.Parser(programtokens)
                _ = parser.parse_member_declaration()
                dataset.append([fp, pc, text, ext, 1])
            except:
                try:
                    programtokens = javalang.tokenizer.tokenize(text)
                    _ = javalang.parser.parse(programtokens)
                    dataset.append([fp, pc, text, ext,1])
                except:
                    dataset.append([fp, pc, text, ext, 0])

paths = ['./data_original/', './data_transpiled/']
for path in paths:
    remove_unnecessary(path)
    erroneous_files = []
    unparsable = []
    dataset = []
    flag = 0
    for (rt,dr,files) in os.walk(path, topdown=True):
            for file in files:
                #print(file)
                ext = file.split('.')[1]
                f_path=rt+'/'+file
                #print(f_path)
                if ext=='py' or ext=='java':
                    is_parsable(f_path, ext)
    print(len(erroneous_files), len(dataset))
    print(len(unparsable), len(dataset)/(len(dataset) + len(unparsable)))
    
    df = pd.DataFrame(dataset, columns=['file_path','problem_class', 'code', 'language_extension', 'is_parsable'])
    if path =='./data_transpiled/':
        or_all = pd.read_pickle(storage_root+'data_original.pkl')
        offset = max(or_all['id'])
        df['is_translated']=1
        
        new_trans = []
        for index, row in df.iterrows():
            fp = row.file_path.split('.')[0]
            ex = row.language_extension
            #i = row.old_id
            if ex=='py':
                tex = fp+'.java'
                
            else:
                tex = fp+'.py'
            
            orid = or_all[or_all.file_path == tex].id.values[0]
            #print(tex,i, orid)
            
            k = list(row)
            #k.append(orid)
            k.append(orid+offset)
            new_trans.append(k)
            #break
            # 'old_id', 'org_id', 
        #df['id'] = [x+offset+1 for x in range(len(df))]
        df = pd.DataFrame(new_trans, columns=['file_path', 'problem_class', 'code', 'language_extension', 'is_parsable', 'is_translated','id'])
        
    else:
        df['is_translated']=0   
        df['id']= df.index+1
    df.to_pickle(str(path[2:-1])+'.pkl')

#testing
or_all = pd.read_pickle(storage_root+'data_original.pkl')
tr_all = pd.read_pickle(storage_root+'data_transpiled.pkl')
print(min(or_all['id']), max(or_all['id']), min(tr_all['id']), max(tr_all['id']))
#print(or_all.head(3))
#print(tr_all.head(3))